# TODO:

1. Load 2014 - 2020 data into a pandas DataFrame
3. Calculate 15 different technical indicators with 15 different period lengths for each oservation
4. Convert the 225(15x15) new features into a 15x15 image
5. Label the data as buy/sell/hold based on trading rules
6. Train Convolutional Neural Network 

In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('2014_to_march2020_btc_minutes_data')
df.shape

(2602679, 6)

In [4]:
df2 = df[:1301342]

In [5]:
df2.shape

(1301342, 6)

In [6]:
del df

In [7]:
df2.head()

,time,open,close,high,low,volume
0,2014-01-01 08:01:00,746.99,747.00,747.00,746.99,2.000000
1,2014-01-01 08:02:00,743.89,744.89,744.89,743.89,0.020695
2,2014-01-01 08:04:00,745.01,745.00,745.01,745.00,0.900000
3,2014-01-01 08:05:00,747.00,747.30,747.30,747.00,2.500000
4,2014-01-01 08:07:00,744.87,744.87,744.87,744.87,1.230769


# Technical Indicators

1. Relative Strength Index (RSI) --> Oscilates between 0 and 100, if over 70 = overbought region, if under 30 = oversold region
2. Williams %R --> Momentum indicator, oscilates between -100 and 0, under -80 = oversold, over -20 = overbought
3. Simple Moving Average (SMA) --> moving average of price for a given period, intersections of SMAs of different periods (MA crosses) used to identify trends
4. Exponential Moving Average (EMA) --> moving average for a period where most recent data has more weight
5. Weighted Moving Average (WMA) --> similar to above EMA except that for WMA importance of closing price decreases lineraly when moving back where EMA decreases exponentially
6. Hull Moving Average (HMA) --> type of moving average to reduce lag associated with SMA, tends to perform better than EMA and WMA
7. Triple Exponential Moving Average (TEMA) --> type of EMA indicator that reduces minor price fluctuations and filters out volatitity
8. Commodity Channel Index (CCI) --> compares current price and average price over a period of time, oscillates about 75% between -100 and 100 and 25% it passes its range values
9. Chande Momentum Oscilator Indicator (CMO) --> momentum similar to RSI and Stochastic Oscillator, oscillates between -100 and 100, if over 50 = over bought and under -50 = oversold
10. Moving Average Convergence and Divergence (MACD) --> shows trends and made up of two parts, MACD and signal line, if MACD crosses signal upward then buy and if downward sell
11. Percentage Price Oscilator (PPO) --> very similar to MACD
12. Rate of Change (ROC) --> illustrates the speed of price change over a period of time
13. Chaikin Money Flow Indicator (CMFI) --> measure money flow volume over a period of time, fluctuates between 1 and -1, closer to 1 = higher buying pressure, -1 = selling pressure higher
14. Directional Movement Indicator (DMI) --> shows trend stregth and direction, made up of 3 seperate indicators, Average Directional Index (ADX), Plus Directional Indicator (+DI), Minus Directional Indicator (-DI)
15. Parabolic SAR (SAR) --> try to determine points of potential stops and reverses, 3 emelements, Previous SAR (PSAR), Extreme Point (EP), and Acceleration Factor (AF)

In [8]:
def get_sma(df, intervals):
    """
    simple moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting sma')
    start_time = datetime.now()
    for period in intervals:
        df["SMA_" + str(period)] = df['close'].rolling(window=period).mean()
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_ema(df, intervals):
    """
    Exponential moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting ema')
    start_time = datetime.now()
    
    for period in intervals:
        df["EMA_" + str(period)] = df['close'].ewm(span=period).mean()
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_wma(df, intervals):
    """
    weighted moving average of price for a given period or interval, used for identifing trends.
    
    df: Pandas DataFrame
    col_name: string, Column in DataFrame that we want to calculate the average on
    intervals: list of ints, list of time periods
    
    return: DataFrame with new columns for each period in list
    """
    print('starting wma')
    start_time = datetime.now()
    
    for period in intervals:
        weights = np.arange(1, period + 1)        
        df["WMA_" + str(period)] = df['close'].rolling(window=period).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_hma(df, intervals, wma_cols=True):
    """
    Hull Moving Average, type of moving average that reduces lag.
    First, calculate the Weighted Moving Average for the period on 'close' price.
    Second, calculate the weighted Moving Average for n/2 periods on 'close' price and multiple the result by 2.
    Third, calculate the Weighted Moving Average for sqrt(n) periods using the values from step 2.
    
    df: Pandas DataFrame, should have Open, High, Lose, Close price data but only Close is needed
    intervals: list of ints, list of time periods to calculate
    wma_cols: Default is False, check to see if Weighted Moving Averages are already calculated, if so use those values for step 1, if not calculate them.
    
    return: Pandas DataFrame df with new Hull Moving Average columns added for each period in intervals list.
    """
    print('starting hull')
    start_time = datetime.now()
    for period in intervals:
        #  wma for period using wma_columns already in DataFrame
        if wma_cols is True:
            wma = df["WMA_" + str(period)]
        # if wma's have not been calculated yet calculate them 
        else:
            # set weights for period
            weights1 = np.arange(1, period + 1)        
            wma1 = df['close'].rolling(window=period).apply(lambda x: np.dot(x, weights1) / weights1.sum(), raw=True)


        # calculate wma for period/2
        # set weights for period/2
        weights2 = np.arange(1, int(period/2)+1)
        wma2 = df['close'].rolling(window=int(period/2)).apply(lambda x: np.dot(x, weights2) / weights2.sum(), raw=True)
        # multiple wma2 by 2
        wma2 = wma2.apply(lambda x: 2*x)

        # subtract wma2 - wma
        wma3 = wma2 - wma

        # calculate wma for sqrt(period) using wma
        # calculate weights for sqrt(period)
        weights3 = np.arange(1, int(np.sqrt(period))+1)
        # calculate Hull Moving Average 
        df["HMA_" + str(period)] = wma3.rolling(window=int(np.sqrt(period))).apply(lambda x: np.dot(x, weights3) / weights3.sum(), raw=True)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))    
    return df


def get_tema(df, intervals):
    """
    Triple Exponential Moving Average (TEMA), type of moving average that tries to filter out volatility and reduce impact of minor price fluctuations.
    
    df: Pandas DataFrame
    col_name: string, Column in DataFrame that we want to calculate the average on
    intervals: list of ints, list of time periods to calculate
    
    return: Pandas DataFrame df with new Triple Exponential Moving Average columns added for each period in intervals list. 
    """
    print('starting tema')
    start_time = datetime.now()
    # formula = (3 * EMA1) - (3 * EMA2) + EMA3
    # EMA1 = Exponential Moving Average (EMA)
    # EMA2 = EMA of EMA1
    # EMA3 = EMA of EMA2

    for period in intervals:
        ema1 = df['close'].ewm(span=period, adjust=False).mean()
        ema2 = ema1.ewm(span=period, adjust=False).mean()
        ema3 = ema2.ewm(span=period, adjust=False).mean()

        df['TEMA_' + str(period)] = ( 3 * ema1 ) - ( 3 * ema2 ) + ema3
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_rsi(df, intervals, sma=True):
    """
    Oscillator type indicator that shows historical strength and weakness of price, will oscillate between 0 and 100
    Typically if greater than 70 considered overbought, if under 30 considered oversold
    
    df: Pandas DataFrame 
    col_name: string, Column name to calculate RSI on
    intervals: list of ints, list of time periods
    
    return: Copy of original DataFrame with new columns for each period in the list
    """
    print('starting rsi')
    start_time = datetime.now()
    # create series of price differences of the chosen column
    delta = df['close'].diff()

    # split up and down
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    # default is simple moving average to get average gains and losses
    if sma is True:
        for period in intervals:
            roll_up = up.rolling(window=period).mean()
            roll_down = down.abs().rolling(window=period).mean()
            RS = roll_up / roll_down
            df["RSI_" + str(period)] = 100.0 - (100 / (1.0 + RS))
            
    # if sma is False use exponential moving average to calculate average gains and losses
    else:
        for period in intervals:
            roll_up = up.ewm(span=period).mean()
            roll_down = down.abs().ewm(span=period).mean()
            RS = roll_up / roll_down
            df["RSI_" + str(period)] = 100.0 - (100 / (1.0 + RS))
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    # Return copy of original dataframe with new columns for RSI
    return df


def get_will_r(df, intervals):
    """
    Momentum oscillator indicator to determine overbought and underbough conditions
    Ranges from -100 to 0, below -80 = oversold, above -20 = overbought
    
    df: Pandas DataFrame, must have high, low and close price data
    intervals: list of ints, list of time periods
    
    return: Copy of original DataFrame with new columns for each period in the list
    """
    print('starting williams_r')
    start_time = datetime.now()
    
    # get indicator for each time period in list of intervals
    for period in intervals:
        # get highest high during period
        highest = df['high'].rolling(window=period).max()

        # get lowest low for period
        lowest = df['low'].rolling(window=period).min()

        # get last close
        last_close = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)

        # williams_r formula
        # (highest high / last close) / (Highest High / lowest low) * -100
        df["williamsR_" + str(period)] = ((highest - last_close) / (highest - lowest)) * -100
        
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_cci(df, intervals, constant=0.015):
    """
    Commodity Channel Index (CCI), compares current prices and the average price over a period of time.
    About 75% of the time it oscillates between -100 and 100, 25% of the time it passes out of this range.
    
    Equation:
    typical price = (high + low + close ) / 3
    ( typical price - 20 period sma of typical price ) / ( constant * mean deviation )
    
    df: Pandas DataFrame, need high, low and close prices
    intervals: list of ints, list of time periods
    constant: int, defalut = 0.015, constant is multiplied by mean deviation
    
    return: df, Pandas DataFrame with new CCI columns for the time periods in the interval list
    """
    print('starting CCI')
    start_time = datetime.now()
    for period in intervals:
        # calculate typical price
        tp = (df['high'] + df['low'] + df['close']) / 3
        
        # use typical price to calculate CCI
        df['CCI_' + str(period)] = pd.Series((tp - tp.rolling(window=period).mean()) / (constant * tp.rolling(window=period).std()))
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))  
    return df

def get_cmo(df, intervals):
    """
    Chande Momentum Oscillator Indicator (CMO)
    Oscillates between -100 and 100, over 50 = over bought, under -50 = over sold
    Equation: ( (sum of up - sum of down) / (sum of up + sum of down) ) * 100
    
    df: Pandas DataFrame
    intervals, list of ints, list of periods
    
    return: df, Pandas DataFrame with new CMO columns for each period in intervals
    """
    print('starting CMO')
    start_time = datetime.now()
    delta = df['close'].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    for period in intervals:
        up_sum = up.rolling(window=period).sum()
        down_sum = down.abs().rolling(window=period).sum()

        df['CMO_' + str(period)] = ( (up_sum - down_sum) / (up_sum + down_sum)) * 100
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df

def get_macd(df, intervals):
    """
    Moving Average Convergence and Divergence (MACD)
    Two equations:
    MACD line = ( 12 period ema - 26 period ema )
    Signal Line = 9 period ema of MACD Line

    try: get ema for intervals, then on each interval calculate the MACD Line and Signal Line
    then calculate the difference between MACD Line and Signal Line
    if MACD Line crosses Signal Line in upward direction = buy, if cross from downward direction = sell
    
    df: Pandas DataFrame, needs to have closing price data
    intervals: list of ints, list of periods
    
    return: df, Pandas DataFrame with MACD_diff columns for each period in the list of intervals
    """
    print('starting MACD')
    start_time = datetime.now()
    for period in intervals:
        ema_n = df['close'].ewm(span=period, adjust=False).mean()
        ema_12 = ema_n.ewm(span=12, adjust=False).mean()
        ema_26 = ema_n.ewm(span=26, adjust=False).mean()

        macd = (ema_12 - ema_26)

        signal = macd.ewm(span=9, adjust=False).mean()

        df['MACD_dif_' + str(period)] = macd - signal
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_ppo(df, intervals):
    """
    Percentage Price Oscillator (PPO), similar to MACD
    Two equations:
    PPO = ( (12 period ema - 26 period ema) / 26 period ema ) * 100
    Signal Line = 9 period ema of PPO

    try: get ema for intervals, then on each interval calculate the PPO and Signal Line
    then calculate the difference between PPO and Signal Line
    if PPO crosses Signal Line in upward direction = buy, if cross from downward direction = sell
    
    df: Pandas DataFrame, needs to have closing price data
    intervals: list of ints, list of periods
    
    return: df, Pandas DataFrame with PPO_diff columns for each period in the list of intervals
    """
    print('starting PPO')
    start_time = datetime.now()
    for period in intervals:
        ema_n = df['close'].ewm(span=period, adjust=False).mean()
        ema_12 = ema_n.ewm(span=12, adjust=False).mean()
        ema_26 = ema_n.ewm(span=26, adjust=False).mean()

        ppo = ((ema_12 - ema_26) / ema_26) * 100

        signal = ppo.ewm(span=9, adjust=False).mean()

        df['PPO_dif_' + str(period)] = ppo - signal
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_roc(df, intervals):
    """
    Rate of Change (ROC), indicates the speed of price change over a period of time.
    equation: ( (Latest Close - Previous Close) / Previous Close ) * 100
    
    df: Pandas DataFrame, needs closing price
    intervals: list of ints, list of periods to calculate ROC
    
    return: df, Pandas DataFrame with new columns for each period in list of intervals
    """
    print('starting ROC')
    start_time = datetime.now()
    for period in intervals:
        previous = df['close'].rolling(window=period).apply(lambda x: x[0], raw=True)
        latest = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)
        df["ROC_" + str(period)] = ( (latest - previous) / previous ) * 100
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time)) 
    return df


def get_cmf(df, intervals):
    """
    Chaikin Money Flow Indicator (CMFI) fluctuates between -1 and 1
    measure money flow volume over a period of time
    equations: 3 parts
    Multiplier = ( (close - low) - (high - close) ) / (high - low)
    Money Flow Volume (MFV) = Volume * Multiplier
    n period CMF = (n period sum of MFV) / ( n period sum of Volume)
    
    df: Pandas DataFrame, needs high, low, close and volume data
    intervals: list of ints, list of periods to calculate CMF on
    
    return: df, Pandas DataFrame with new columns for each period in list of intervals
    """
    print('starting CMF')
    start_time = datetime.now()
    for period in intervals:
        multiplier = (df['close'] - df['low']) - (df['high'] - df['close']) / (df['high'] - df['low'])
        money_flow_volume = df['volume'] * multiplier
        df['CMF_' + str(period)] = money_flow_volume.rolling(window=period).sum() / df['volume'].rolling(window=period).sum()
        df['CMF_' + str(period)].fillna(0, inplace=True)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    return df


def get_atr(df, intervals):
    """
    Average True Range measures market volatility, first solve true range then take an average of true range over period n
    
    True range: take the values below and find the maximum
    1. current high - previous low
    2. absolute value of current high - previous close
    3. absolute value of current low - previous close
    
    Once we have true range find the average for period n
    
    df: Pandas DataFrame with open, high, low, close, volume data
    intervals: list of integers, list of periods to calculate averages on
    
    return: Pandas DataFrame with new average true range column for each period in intervals list
    """
    print('starting atr')
    start_time = datetime.now()
    
    for i in range(df.index[0], len(df)):
        if i == df.index[0]:
            pass
        else:
            candle_range = df.at[i, 'high'] - df.at[i,'low']
            high_close_range = abs(df.at[i,'high'] - df.at[i-1,'close'])
            low_close_range = abs(df.at[i,'low'] - df.at[i-1,'close'])
            
            df.at[i,'true_range'] = max(candle_range, high_close_range, low_close_range)
    
    for period in intervals:
        df['avg_tr_'+str(period)] = df['true_range'].rolling(window= period).mean()
        
    df.drop(columns='true_range', inplace=True)
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df


def get_stochastic(df, intervals):
    """
    Stochastic Oscillator
    formula ( (current close - lowest low in period) / (highest high in period - lowest low in period) ) * 100
    ranges between 0 - 100 similar to RSI greater than 80 = overbought, less than 20 = oversold
    note: descided to take the exponential moving average of the indicator
    
    df: Pandas DataFrame, needs, open, close, low, high, and volume
    intervals: list of integers, list of time periods to calculate indicator over
    
    return: Pandas DataFrame with new columns for Stochastic Oscillator over the time periods in the list of intervals
    """

    print('starting stochastic')
    start_time = datetime.now()
    
    for period in intervals:
        # get highest high during period
        highest = df['high'].rolling(window=period).max()

        # get lowest low for period
        lowest = df['low'].rolling(window=period).min()

        # get last close
        last_close = df['close'].rolling(window=period).apply(lambda x: x[-1], raw=True)

        # calculate indicator
        oscillator = ((last_close - lowest) / (highest - lowest)) * 100

        # take exponential moving average for each period
        df['stoch_os_'+str(period)] = oscillator.ewm(span=period, adjust=False).mean()
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    return df

In [9]:
# set list of time periods 6 through 20 for 15 periods
intervals = [ i + 6 for i in range(15) ]

In [10]:
def calculate_moving_avg_indicators(df, intervals):
    get_sma(df, intervals)
    get_ema(df, intervals)
    get_wma(df, intervals)
    get_hma(df, intervals)
    get_tema(df, intervals)
        
    return df

In [11]:
def calculat_momentum_indicators(df, intervals):
    get_rsi(df, intervals)
    get_will_r(df, intervals)
    get_cci(df, intervals)
    get_cmo(df, intervals)
    get_macd(df, intervals)
        
    return df

In [12]:
def calculate_last_indicators(df, intervals):
    get_ppo(df, intervals)
    get_roc(df, intervals)
    get_cmf(df, intervals)
    get_atr(df, intervals)
    get_stochastic(df, intervals)
    
    return df

In [13]:
df2 = calculate_moving_avg_indicators(df2, intervals)

starting sma
Duration: 0:00:00.715693
starting ema
Duration: 0:00:01.670743
starting wma
Duration: 0:01:14.536265
starting hull
Duration: 0:02:27.875158
starting tema
Duration: 0:00:05.544710


In [14]:
df2 = calculat_momentum_indicators(df2, intervals)

starting rsi
Duration: 0:00:18.219907
starting williams_r
Duration: 0:00:07.358757
starting CCI
Duration: 0:00:01.647407
starting CMO
Duration: 0:00:17.829662
starting MACD
Duration: 0:00:09.440747


In [15]:
df2 = calculate_last_indicators(df2, intervals)

starting PPO
Duration: 0:00:09.322137
starting ROC
Duration: 0:00:11.545892
starting CMF
Duration: 0:00:02.061200
starting atr
Duration: 0:01:43.954521
starting stochastic
Duration: 0:00:10.373946


In [16]:
df2.shape

(1301342, 231)

In [17]:
df2.tail()

,time,open,close,high,low,volume,SMA_6,SMA_7,SMA_8,SMA_9,SMA_10,SMA_11,SMA_12,SMA_13,SMA_14,SMA_15,SMA_16,SMA_17,SMA_18,SMA_19,SMA_20,EMA_6,EMA_7,EMA_8,EMA_9,EMA_10,EMA_11,EMA_12,EMA_13,EMA_14,EMA_15,EMA_16,EMA_17,EMA_18,EMA_19,EMA_20,WMA_6,WMA_7,WMA_8,WMA_9,WMA_10,WMA_11,WMA_12,WMA_13,WMA_14,WMA_15,WMA_16,WMA_17,WMA_18,WMA_19,WMA_20,HMA_6,HMA_7,HMA_8,HMA_9,HMA_10,HMA_11,HMA_12,HMA_13,HMA_14,HMA_15,HMA_16,HMA_17,HMA_18,HMA_19,HMA_20,TEMA_6,TEMA_7,TEMA_8,TEMA_9,TEMA_10,TEMA_11,TEMA_12,TEMA_13,TEMA_14,TEMA_15,TEMA_16,TEMA_17,TEMA_18,TEMA_19,TEMA_20,RSI_6,RSI_7,RSI_8,RSI_9,RSI_10,RSI_11,RSI_12,RSI_13,RSI_14,RSI_15,RSI_16,RSI_17,RSI_18,RSI_19,RSI_20,williamsR_6,williamsR_7,williamsR_8,williamsR_9,williamsR_10,williamsR_11,williamsR_12,williamsR_13,williamsR_14,williamsR_15,williamsR_16,williamsR_17,williamsR_18,williamsR_19,williamsR_20,CCI_6,CCI_7,CCI_8,CCI_9,CCI_10,CCI_11,CCI_12,CCI_13,CCI_14,CCI_15,CCI_16,CCI_17,CCI_18,CCI_19,CCI_20,CMO_6,CMO_7,CMO_8,CMO_9,CMO_10,CMO_11,CMO_12,CMO_13,CMO_14,CMO_15,CMO_16,CMO_17,CMO_18,CMO_19,CMO_20,MACD_dif_6,MACD_dif_7,MACD_dif_8,MACD_dif_9,MACD_dif_10,MACD_dif_11,MACD_dif_12,MACD_dif_13,MACD_dif_14,MACD_dif_15,MACD_dif_16,MACD_dif_17,MACD_dif_18,MACD_dif_19,MACD_dif_20,PPO_dif_6,PPO_dif_7,PPO_dif_8,PPO_dif_9,PPO_dif_10,PPO_dif_11,PPO_dif_12,PPO_dif_13,PPO_dif_14,PPO_dif_15,PPO_dif_16,PPO_dif_17,PPO_dif_18,PPO_dif_19,PPO_dif_20,ROC_6,ROC_7,ROC_8,ROC_9,ROC_10,ROC_11,ROC_12,ROC_13,ROC_14,ROC_15,ROC_16,ROC_17,ROC_18,ROC_19,ROC_20,CMF_6,CMF_7,CMF_8,CMF_9,CMF_10,CMF_11,CMF_12,CMF_13,CMF_14,CMF_15,CMF_16,CMF_17,CMF_18,CMF_19,CMF_20,avg_tr_6,avg_tr_7,avg_tr_8,avg_tr_9,avg_tr_10,avg_tr_11,avg_tr_12,avg_tr_13,avg_tr_14,avg_tr_15,avg_tr_16,avg_tr_17,avg_tr_18,avg_tr_19,avg_tr_20,stoch_os_6,stoch_os_7,stoch_os_8,stoch_os_9,stoch_os_10,stoch_os_11,stoch_os_12,stoch_os_13,stoch_os_14,stoch_os_15,stoch_os_16,stoch_os_17,stoch_os_18,stoch_os_19,stoch_os_20
1301337,2017-09-20 14:09:00,4003.2,4015.3,4015.3,4003.1,27.877553,4005.183333,4001.585714,4000.0250,3999.944444,4000.43,4000.736364,4001.283333,4001.800000,4003.100000,4002.413333,4000.50625,3997.900000,3994.955556,3992.300000,3989.680,4004.962503,4003.969413,4002.971694,4001.935707,4000.850230,3999.716582,3998.542250,3997.337145,3996.111537,3994.875016,3993.636037,3992.401781,3991.178181,3989.970031,3988.781120,4006.419048,4005.210714,4004.058333,4003.235556,4002.725455,4002.393939,4002.223077,4002.162637,4002.287619,4002.303333,4002.091912,4001.626144,4000.923977,4000.061579,3999.072857,4009.026984,4010.166270,4009.207963,4008.879259,4008.077273,4008.196212,4006.585531,4006.473932,4004.270476,4004.374544,4002.363734,4003.059935,4003.140959,4004.153848,4004.892009,4010.769522,4009.756668,4009.083959,4008.801276,4008.873726,4009.228585,4009.785200,4010.469830,4011.221234,4011.991493,4012.744718,4013.455065,4014.104691,4014.681956,4015.179923,87.157895,73.144876,61.976048,57.261411,57.844475,55.208333,54.709677,47.374302,59.640103,65.770349,69.275930,71.385176,71.454545,72.228774,72.440023,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-9.580838,-12.548263,-30.414747,-25.714286,-21.951220,-21.734358,-20.798319,-20.183486,116.393601,71.530129,80.772957,86.741872,86.845991,85.829286,83.565412,79.222901,73.483036,77.549352,68.399611,60.027672,57.602812,58.367389,59.798782,74.315789,46.289753,23.952096,14.522822,15.688950,10.416667,9.419355,-5.251397,19.280206,31.540698,38.551859,42.770352,42.909091,44.457547,44.880047,0.255568,0.477155,0.686472,0.876677,1.045289,1.192298,1.318962,1.427101,1.518711,1.595750,1.660040,1.713230,1.756778,1.791966,1.819912,0.005577,0.011168,0.016458,0.021273,0.025549,0.029285,0.032512,0.035274,0.037620,0.039600,0.041258,0.042636,0.043770,0.044692,0.045430,0.289732,0.886935,0.656790,0.400070,0.262185,0.287227,0.199636,0.182136,-0.116915,0.563514,1.092676,1.493858,1.784583,1.794904,1.913754,16.681344,12.703981,11.245156,10.912806,10.664200,10.066419,9.543323,9.479202,11.766042,16.553632,16.172537,15.801182,15.64

In [18]:
df2.head(25)

,time,open,close,high,low,volume,SMA_6,SMA_7,SMA_8,SMA_9,SMA_10,SMA_11,SMA_12,SMA_13,SMA_14,SMA_15,SMA_16,SMA_17,SMA_18,SMA_19,SMA_20,EMA_6,EMA_7,EMA_8,EMA_9,EMA_10,EMA_11,EMA_12,EMA_13,EMA_14,EMA_15,EMA_16,EMA_17,EMA_18,EMA_19,EMA_20,WMA_6,WMA_7,WMA_8,WMA_9,WMA_10,WMA_11,WMA_12,WMA_13,WMA_14,WMA_15,WMA_16,WMA_17,WMA_18,WMA_19,WMA_20,HMA_6,HMA_7,HMA_8,HMA_9,HMA_10,HMA_11,HMA_12,HMA_13,HMA_14,HMA_15,HMA_16,HMA_17,HMA_18,HMA_19,HMA_20,TEMA_6,TEMA_7,TEMA_8,TEMA_9,TEMA_10,TEMA_11,TEMA_12,TEMA_13,TEMA_14,TEMA_15,TEMA_16,TEMA_17,TEMA_18,TEMA_19,TEMA_20,RSI_6,RSI_7,RSI_8,RSI_9,RSI_10,RSI_11,RSI_12,RSI_13,RSI_14,RSI_15,RSI_16,RSI_17,RSI_18,RSI_19,RSI_20,williamsR_6,williamsR_7,williamsR_8,williamsR_9,williamsR_10,williamsR_11,williamsR_12,williamsR_13,williamsR_14,williamsR_15,williamsR_16,williamsR_17,williamsR_18,williamsR_19,williamsR_20,CCI_6,CCI_7,CCI_8,CCI_9,CCI_10,CCI_11,CCI_12,CCI_13,CCI_14,CCI_15,CCI_16,CCI_17,CCI_18,CCI_19,CCI_20,CMO_6,CMO_7,CMO_8,CMO_9,CMO_10,CMO_11,CMO_12,CMO_13,CMO_14,CMO_15,CMO_16,CMO_17,CMO_18,CMO_19,CMO_20,MACD_dif_6,MACD_dif_7,MACD_dif_8,MACD_dif_9,MACD_dif_10,MACD_dif_11,MACD_dif_12,MACD_dif_13,MACD_dif_14,MACD_dif_15,MACD_dif_16,MACD_dif_17,MACD_dif_18,MACD_dif_19,MACD_dif_20,PPO_dif_6,PPO_dif_7,PPO_dif_8,PPO_dif_9,PPO_dif_10,PPO_dif_11,PPO_dif_12,PPO_dif_13,PPO_dif_14,PPO_dif_15,PPO_dif_16,PPO_dif_17,PPO_dif_18,PPO_dif_19,PPO_dif_20,ROC_6,ROC_7,ROC_8,ROC_9,ROC_10,ROC_11,ROC_12,ROC_13,ROC_14,ROC_15,ROC_16,ROC_17,ROC_18,ROC_19,ROC_20,CMF_6,CMF_7,CMF_8,CMF_9,CMF_10,CMF_11,CMF_12,CMF_13,CMF_14,CMF_15,CMF_16,CMF_17,CMF_18,CMF_19,CMF_20,avg_tr_6,avg_tr_7,avg_tr_8,avg_tr_9,avg_tr_10,avg_tr_11,avg_tr_12,avg_tr_13,avg_tr_14,avg_tr_15,avg_tr_16,avg_tr_17,avg_tr_18,avg_tr_19,avg_tr_20,stoch_os_6,stoch_os_7,stoch_os_8,stoch_os_9,stoch_os_10,stoch_os_11,stoch_os_12,stoch_os_13,stoch_os_14,stoch_os_15,stoch_os_16,stoch_os_17,stoch_os_18,stoch_os_19,stoch_os_20
0,2014-01-01 08:01:00,746.99,747.0000,747.0000,746.9900,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01 08:02:00,743.89,744.8900,744.8900,743.8900,0.020695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.769167,745.794286,745.813125,745.827778,745.839500,745.849091,745.857083,745.863846,745.869643,745.874667,745.879063,745.882941,745.886389,745.889474,745.892250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.658950,745.780156,745.882771,745.970320,746.045665,746.111065,746.168293,746.218746,746.263532,746.303535,746.339471,746.371920,746.401362,746.428190,746.452735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
df2.to_csv("top_half_indicators")